In [83]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
from model import *
from data import prepare_data_vocab, live_feat
from torch.nn.utils.rnn import unpack_sequence
from torch_geometric.data import Data


import torch.optim as optim
from torch_geometric.data import Data

In [85]:
word_to_idx, idx_to_word, data, data_emb = prepare_data_vocab("data", func=live_feat, function_num=0)

{0: {'static': {}, 'temp': {}}, 1: {'static': {}, 'temp': {}}, 2: {'static': {}, 'temp': {}}, 3: {'static': {}, 'temp': {}}, 4: {'static': {}, 'temp': {}}}
{0: {'static': {}, 'temp': {}}, 1: {'static': {}, 'temp': {}}, 2: {'static': {}, 'temp': {}}, 3: {'static': {}, 'temp': {}}, 4: {'static': {}, 'temp': {}}}
{0: {'static': {}, 'temp': {}}, 1: {'static': {}, 'temp': {}}, 2: {'static': {}, 'temp': {}}, 3: {'static': {}, 'temp': {}}, 4: {'static': {}, 'temp': {}}}
{0: {'static': {}, 'temp': {}}, 1: {'static': {}, 'temp': {}}, 2: {'static': {}, 'temp': {}}, 3: {'static': {}, 'temp': {}}, 4: {'static': {}, 'temp': {}}}
{0: {'static': {}, 'temp': {}}, 1: {'static': {}, 'temp': {}}, 2: {'static': {}, 'temp': {}}, 3: {'static': {}, 'temp': {}}, 4: {'static': {}, 'temp': {}}}
{0: {'static': {'vstate': 'void *'}, 'temp': {}}, 1: {'static': {'vstate': 'void *'}, 'temp': {}}, 2: {'static': {'vstate': 'void *', 's': 'long unsigned int'}, 'temp': {}}}
{0: {'static': {'vstate': 'void *'}, 'temp': {

In [91]:
#define InputEncoder (RNN) and GNN
in_dim = 8
hidden_dim = 64
num_layers = 7
bidirectional = True
out_dim    = 128

int_dir = 2 if bidirectional else 1

bb_enc = InputEncoder(word_to_idx, 
                      in_dim=in_dim, 
                      hidden_dim=hidden_dim, 
                      num_layers=num_layers, 
                      bidirectional=bidirectional)
 
gnn = GNNModel(in_dim=int_dir*hidden_dim,
               out_dim=out_dim,
               layer_dims=[16, 32])

In [101]:
in_dim = 8
hidden_dim = 16
num_layers = 7
bidirectional = True
out_dim    = 32

int_dir = 2 if bidirectional else 1
integrated = IntegratedModel(word_to_idx, 
                      in_dim_lstm=in_dim, 
                      hidden_dim=hidden_dim, 
                      num_layers=num_layers, 
                      bidirectional=bidirectional,
                        out_dim=out_dim,
                        layer_dims=[16, 32])

In [6]:
assert len(data)==len(data_emb)
for i in range(len(data)):
    assert len(data[i][2])==len(data_emb[i][1])

#define InputEncoder (RNN) and GNN
in_dim = 8
hidden_dim = 16
num_layers = 7
bidirectional = True
out_dim    = 16

int_dir = 2 if bidirectional else 1

bb_enc = InputEncoder(word_to_idx, 
                      in_dim=in_dim, 
                      hidden_dim=hidden_dim, 
                      num_layers=num_layers, 
                      bidirectional=bidirectional)
 
gnn = GNNModel(in_dim=int_dir*hidden_dim,
               out_dim=out_dim,
               layer_dims=[16, 32])

In [38]:
initial_node_embeddings = []
new_data_emb = []
for graph in data_emb:
    edge_index, node_embs = graph
    #if torch.tensor(edge_index).shape[0]==0:
    if len(edge_index) != 0:
        edge_index = torch.tensor(edge_index).T
    else:
        edge_index = torch.tensor([[0],[0]])
    
    out, h, c = bb_enc(node_embs)
    node_feats = torch.vstack([k[-1] for k in unpack_sequence(out)])

    assert node_feats.shape == (len(node_embs), int_dir*hidden_dim)

            #Step 2: gnn forward pass
    graph = Data(edge_index=edge_index,
                        x=node_feats)
    initial_node_embeddings.append(torch.rand((graph.num_nodes, out_dim), requires_grad=True))
    new_data_emb.append([edge_index, node_embs, initial_node_embeddings])

In [28]:
edge_index, node_embs = data_emb[0]
#if torch.tensor(edge_index).shape[0]==0:

if len(edge_index) != 0:
    edge_index = torch.tensor(edge_index).T
else:
    edge_index = torch.tensor([[0],[0]])
            
        
out, h, c = bb_enc(node_embs)

In [30]:
out

PackedSequence(data=tensor([[ 0.0196, -0.0244, -0.0206,  ...,  0.0122,  0.0122, -0.0505],
        [ 0.0196, -0.0243, -0.0206,  ...,  0.0122,  0.0119, -0.0505],
        [ 0.0180, -0.0271, -0.0214,  ...,  0.0167,  0.0171, -0.0428],
        ...,
        [ 0.0216, -0.0767, -0.0376,  ...,  0.0159,  0.0168, -0.0404],
        [ 0.0170, -0.0873, -0.0395,  ...,  0.0135,  0.0149, -0.0244],
        [ 0.0171, -0.0872, -0.0395,  ...,  0.0135,  0.0148, -0.0244]],
       grad_fn=<CatBackward0>), batch_sizes=tensor([14,  6,  6,  2,  2]), sorted_indices=tensor([ 0,  6,  2,  4,  8, 10,  1,  3,  5,  7,  9, 11, 12, 13]), unsorted_indices=tensor([ 0,  6,  2,  7,  3,  8,  1,  9,  4, 10,  5, 11, 12, 13]))

In [102]:
import random
import copy
criterion = nn.MSELoss()
optimizer = optim.Adam(integrated.parameters(), lr=0.1)

epoch_num = 100
# Training loop



for epoch in range(epoch_num):
    total_loss = 0
    total_items = 0
    j = -1
    Min = 0
    Max = 0
    Mean = 0
    for graph in data_emb:
        optimizer.zero_grad()
        j += 1
        edge_index, node_embs = graph
        #if torch.tensor(edge_index).shape[0]==0:
        if len(edge_index) != 0:
            edge_index = torch.tensor(edge_index).T
        else:
            edge_index = torch.tensor([[0],[0]])
            
        
       
        #random_indices = random.sample(range(0, 128), 8)
        random_indices = [10*x for x in range((int)(int_dir*hidden_dim/10))]        
        

        gnn_output, node_feats = integrated(node_embs, edge_index)
        
        assert node_feats.shape == (len(node_embs), int_dir*hidden_dim)
       
        
        
        # Unsupervised loss (dummy loss for illustration)
        #loss = criterion(output, random_target[j])
        loss = criterion(gnn_output, node_feats)

        # Backward pass and optimization
        total_loss += loss.item()
        total_items += node_feats.shape[0]

        
        
        loss.backward()
        optimizer.step()

        Min += node_feats.min()/len(node_feats)
        Mean += node_feats.mean()/len(node_feats)
        Max += node_feats.max()/len(node_feats)
        
        if epoch%2==0 or epoch==epoch_num-1:
            avg_loss = total_loss / (j+1)
            print(f"Min: {Min:.10f}, Mean: {Mean:.10f}, Max: {Max:.10f}", end=' |#| ')
            print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {avg_loss:.10f}', end='\r')

        #total_loss += unsupervised_loss.item()

    # Print or log the average loss for monitoring
    if epoch%2==0 or epoch==epoch_num-1:
        print()


Min: -125.4231643677, Mean: 2.4671511650, Max: 135.5606994629 |#| For 578 functions: Epoch 1/100, Average Loss: 0.0018071440
Min: -126.2934646606, Mean: 2.5309195518, Max: 136.0403289795 |#| For 578 functions: Epoch 3/100, Average Loss: 0.0006994761
Min: -126.2934646606, Mean: 2.5309195518, Max: 136.0403289795 |#| For 578 functions: Epoch 5/100, Average Loss: 0.0007130851
Min: -126.2934646606, Mean: 2.5309195518, Max: 136.0403289795 |#| For 578 functions: Epoch 7/100, Average Loss: 0.0007184432
Min: -126.2934646606, Mean: 2.5309195518, Max: 136.0403289795 |#| For 578 functions: Epoch 9/100, Average Loss: 0.0007213074
Min: -126.2934646606, Mean: 2.5309195518, Max: 136.0403289795 |#| For 578 functions: Epoch 11/100, Average Loss: 0.0007218685
Min: -126.2934646606, Mean: 2.5309195518, Max: 136.0403289795 |#| For 578 functions: Epoch 13/100, Average Loss: 0.0007220605
Min: -126.2934646606, Mean: 2.5309195518, Max: 136.0403289795 |#| For 578 functions: Epoch 15/100, Average Loss: 0.00072195

KeyboardInterrupt: 

In [99]:
criterion = nn.MSELoss()
optimizer = optim.Adam(gnn.parameters(), lr=0.01)

epoch_num = 300
# Training loop

random_target = torch.rand((len(data_emb), 1, out_dim))

for epoch in range(epoch_num):
    total_loss = 0
    total_items = 0
    j = -1
    for graph in data_emb:
        j += 1
        edge_index, node_embs = graph
        #if torch.tensor(edge_index).shape[0]==0:
        if len(edge_index) != 0:
            edge_index = torch.tensor(edge_index).T
        else:
            edge_index = torch.tensor([[0],[0]])
            
        
        out, h, c = bb_enc(node_embs)
        node_feats = torch.vstack([k[-1] for k in unpack_sequence(out)])

        assert node_feats.shape == (len(node_embs), int_dir*hidden_dim)

                #Step 2: gnn forward pass
        graph = Data(edge_index=edge_index,
                            x=node_feats)
        
        output = gnn(graph)
        # Unsupervised loss (dummy loss for illustration)
        loss = criterion(output, random_target[j])
        #loss = criterion(output, initial_node_embeddings[j])

        # Backward pass and optimization
        total_loss  += loss.item()
        total_items += graph.x.shape[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch%10==0 or epoch==epoch_num-1:
            avg_loss = total_loss / len(data_emb)
            print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {avg_loss:.5f}', end='\r')

        #total_loss += unsupervised_loss.item()

    # Print or log the average loss for monitoring
    if epoch%10==0 or epoch==epoch_num-1:
        print()


For 578 functions: Epoch 1/300, Average Loss: 0.08449


KeyboardInterrupt: 

In [61]:
out_act

tensor([[-0.0595, -1.0000,  0.0944,  0.0012, -0.0701,  0.0644, -1.0000,  0.0634,
         -1.0000,  0.0314,  0.0223,  0.0017,  0.0381, -0.0014, -1.0000, -0.0373,
         -0.0400,  0.0077, -0.0531,  0.0995,  0.0402, -0.0712, -0.0021,  0.0273,
          0.0558,  0.0899,  0.0339, -0.0429, -0.0047, -0.0116,  0.0371, -0.0692],
        [-0.0595, -1.0000,  0.0944,  0.0012, -0.0701,  0.0643, -1.0000,  0.0634,
         -1.0000,  0.0313,  0.0223,  0.0017,  0.0381, -0.0014, -1.0000, -0.0373,
         -0.0400,  0.0077, -0.0531,  0.0995,  0.0402, -0.0712, -0.0021,  0.0274,
          0.0558,  0.0899,  0.0339, -0.0429, -0.0047, -0.0116,  0.0371, -0.0692]])

In [98]:
for name, param in bb_enc.named_parameters():
    if param.grad is not None:
        print(f"{name} grad norm: {param.grad.norm()}")
    else:
        print(f"{name} grad is None")

emb.weight grad norm: 4.2484352889005095e-05
lstm.weight_ih_l0 grad norm: 0.0003254949115216732
lstm.weight_hh_l0 grad norm: 7.605506834806874e-05
lstm.bias_ih_l0 grad norm: 0.0003012344823218882
lstm.bias_hh_l0 grad norm: 0.0003012344823218882
lstm.weight_ih_l0_reverse grad norm: 0.00032709658262319863
lstm.weight_hh_l0_reverse grad norm: 7.603189442306757e-05
lstm.bias_ih_l0_reverse grad norm: 0.000313809810904786
lstm.bias_hh_l0_reverse grad norm: 0.000313809810904786
lstm.weight_ih_l1 grad norm: 0.000558680621907115
lstm.weight_hh_l1 grad norm: 0.00022624272969551384
lstm.bias_ih_l1 grad norm: 0.0010487716645002365
lstm.bias_hh_l1 grad norm: 0.0010487715480849147
lstm.weight_ih_l1_reverse grad norm: 0.0004790965758729726
lstm.weight_hh_l1_reverse grad norm: 0.00016279509873129427
lstm.bias_ih_l1_reverse grad norm: 0.0009165560477413237
lstm.bias_hh_l1_reverse grad norm: 0.0009165561059489846
lstm.weight_ih_l2 grad norm: 0.0014243311015889049
lstm.weight_hh_l2 grad norm: 0.000649143

In [100]:
for name, param in bb_enc.named_parameters():
    if param.grad is not None:
        print(f"{name} grad norm: {param.grad.norm()}")
    else:
        print(f"{name} grad is None")

emb.weight grad norm: 4.2484352889005095e-05
lstm.weight_ih_l0 grad norm: 0.0003254949115216732
lstm.weight_hh_l0 grad norm: 7.605506834806874e-05
lstm.bias_ih_l0 grad norm: 0.0003012344823218882
lstm.bias_hh_l0 grad norm: 0.0003012344823218882
lstm.weight_ih_l0_reverse grad norm: 0.00032709658262319863
lstm.weight_hh_l0_reverse grad norm: 7.603189442306757e-05
lstm.bias_ih_l0_reverse grad norm: 0.000313809810904786
lstm.bias_hh_l0_reverse grad norm: 0.000313809810904786
lstm.weight_ih_l1 grad norm: 0.000558680621907115
lstm.weight_hh_l1 grad norm: 0.00022624272969551384
lstm.bias_ih_l1 grad norm: 0.0010487716645002365
lstm.bias_hh_l1 grad norm: 0.0010487715480849147
lstm.weight_ih_l1_reverse grad norm: 0.0004790965758729726
lstm.weight_hh_l1_reverse grad norm: 0.00016279509873129427
lstm.bias_ih_l1_reverse grad norm: 0.0009165560477413237
lstm.bias_hh_l1_reverse grad norm: 0.0009165561059489846
lstm.weight_ih_l2 grad norm: 0.0014243311015889049
lstm.weight_hh_l2 grad norm: 0.000649143

In [51]:
for name, param in integrated.named_parameters():
    if not param.requires_grad:
        print("not", param)
    

AttributeError: 'generator' object has no attribute 'grad'

In [49]:
import re

def extract_a_op_b(input_string):
    

    # Find and replace expressions within parentheses with a placeholder
    placeholder = '__PLACEHOLDER__'
    flag_par = 0
    while '(' in input_string and ')' in input_string:
        flag_par = 1
        # Find the innermost expression within parentheses
        inner_expr = re.search(r'\([^)]*\)', input_string).group(0)
        
        # Replace the inner expression with a placeholder
        input_string = input_string.replace(inner_expr, placeholder)

    # Define the original pattern
    original_pattern = r"([-+]?\b\d+(\.\d*)?(e[+-]?\d+)?\b|\b\w+\b|\[\s*\d+(\s*,\s*\d+)*\s*\])\s*([+\-*/])\s*([-+]?\b\d+(\.\d*)?(e[+-]?\d+)?\b|\b\w+\b|\[\s*\d+(\s*,\s*\d+)*\s*\])"

    # Find the first match in the modified input string
    match = re.search(original_pattern, input_string)

    if match:
        # Extract a, op, b from the match
        a = match.group(1)
        op = match.group(5)
        b = match.group(6)

        # Replace the placeholder back with the original expression
        if flag_par:
            a = a.replace(placeholder, inner_expr)
            b = b.replace(placeholder, inner_expr)

        return a, op, b
    else:
        return None

# Example usage
input_string = "_1 * 5.0e-1"
result = extract_a_op_b(input_string)

if result:
    a, op, b = result
    print("a:", a)
    print("op:", op)
    print("b:", b)
else:
    print("No match found.")


a: _1
op: *
b: 5.0e-1


In [35]:
import numpy as np
vocab = np.array(['a', 'b', 'c', 'a'])
vocab = np.unique(vocab)
word_to_idx = dict(zip(vocab, np.arange(len(vocab))))
idx_to_word = dict(zip(np.arange(len(vocab)), vocab))

In [36]:
word_to_idx

{'a': 0, 'b': 1, 'c': 2}